# About this code
- This is a code for FG2020 ABAW AU detection Challenge.
     - paper title: Action Unit Recognition by Pairwise Deep Architecture
- Copyright 2020 FUJITSU LABORATORIES LTD.

# Libraries to import

In [ ]:
import random
import glob
import os
import shutil
import torch
import pandas as pd
from PIL import Image
from torchvision import models, transforms, datasets
import torch.nn as nn
import torch.optim as optim
import numpy as np
import copy
import matplotlib.pyplot as plt
import json
import scipy
from scipy.stats import gaussian_kde

random.seed(0)
np.random.seed(0)

# Parameters

## Parameters to need to set

In [ ]:
au_name = "AU01"

## Other parameters

In [ ]:
corpus_top_dirname = "../../corpus"
dataset_lst_dirname = "./BinaryData"

phase_lst = ["Train","Valid","Test"]
au_name_lst = [au_name]

train_size_per_epoch = 2048*4
epoch_num = 10
dataset_num_lst = { "Train":train_size_per_epoch*epoch_num, "Valid":512, "Test":512 }
intensity_conv_dataset_num_lst = { "Train":2048*10, "Valid":2048*2, "Test":2048*2, "TestFull":-1, "TrainFull":-1,"ValidFull":-1 }
small_debug_mode = False
intensity_tune_sampling_ratio = 1.0
intensity_tune_sampling_min = 10000000
trial_num = 5


# train_size_per_epoch = 128
# epoch_num = 2
# dataset_num_lst = { "Train":train_size_per_epoch*epoch_num, "Valid":32, "Test":32 }
# intensity_conv_dataset_num_lst = { "Train":128, "Valid":32, "Test":32, "TestFull":-1, "TrainFull":-1,"ValidFull":-1 }
# small_debug_mode = True
# intensity_tune_sampling_ratio = 0.0
# intensity_tune_sampling_min = 10
# trial_num = 2


intensity_net_feature_lst = [ "hist", "percentile" ]

map_location = 'cuda:0'

batch_size_bin = 32
batch_size_int = 32

int_occ_mode = "occ"

In [ ]:
if int_occ_mode == "occ":  

    phase_full_lst = ["ValidFull", "TestFull"]
    
    pose_lst = [1]
    
    dataset_path_lst = {}
    dataset_path_lst["Train"] = {}
    dataset_path_lst["Train"]["posesall"] = {}
    dataset_path_lst["Train"]["posesall"]["FERA"] = [ (corpus_top_dirname + "/FERA/tmp/frames_for_pytorch_procrustes_int_pose%d/Train" % pose) for pose in [1,2,3,4,5,6,7,8,9] ]
    dataset_path_lst["Train"]["posesall"]["FERA"] += [ (corpus_top_dirname + "/FERA_mirror/tmp/frames_for_pytorch_procrustes_int_pose%d/Train" % pose) for pose in [1,2,3,4,5,6,7,8,9] ]
    dataset_path_lst["Train"]["posesall"]["FERA"] += [ (corpus_top_dirname + "/FERA_pose/tmp/frames_for_pytorch_procrustes_int_pose%d/Train" % pose) for pose in [1,2] ]
    dataset_path_lst["Train"]["posesall"]["FERA"] += [ (corpus_top_dirname + "/FERA_pose_mirror/tmp/frames_for_pytorch_procrustes_int_pose%d/Train" % pose) for pose in [1,2] ]
   
    
    dataset_path_lst["Train"]["posesall"]["DISFA"] = [ (corpus_top_dirname + "/DISFA/tmp/frames_for_pytorch_procrustes_int_pose%d/Train" % pose) for pose in [1,2] ]    
    dataset_path_lst["Train"]["posesall"]["AffWild2"] = [ (corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Train" % pose) for pose in pose_lst ]
 
    
    dataset_path_lst["Valid"] = { "pose%d" % pose : {"AffWild2":[(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Valid" % pose)]}  for pose in pose_lst }
    dataset_path_lst["Test"] = { "pose%d" % pose : {"AffWild2":[(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Test" % pose)]}  for pose in pose_lst }


    intensity_tune_dataset_path_lst = {}

    intensity_tune_dataset_path_lst["Train"] = {}
    intensity_tune_dataset_path_lst["Train"]["posesall"] = {}
    intensity_tune_dataset_path_lst["Train"]["posesall"]["AffWild2"] = [ (corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Train" % pose) for pose in pose_lst ]
    

    intensity_tune_dataset_path_lst["Valid"] = { "pose%d" % pose : { "AffWild2": [(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Valid" % pose)]}  for pose in pose_lst }   
    intensity_tune_dataset_path_lst["ValidFull"] = { "pose%d" % pose : { "AffWild2": [(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Valid" % pose)]}  for pose in pose_lst }
    
    intensity_tune_dataset_path_lst["Test"] = { "pose%d" % pose : { "AffWild2": [(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Test" % pose)]}  for pose in pose_lst }    
    intensity_tune_dataset_path_lst["TestFull"] = { "pose%d" % pose : { "AffWild2": [(corpus_top_dirname + "/AffWild2/tmp/frames_for_pytorch_procrustes_occ_pose%d/Test" % pose)]}  for pose in pose_lst }
    
   
    enable_pair_dataset_equal_test = True
    

In [ ]:
device = torch.device(map_location)

# Definition of functions

In [ ]:
def init_dir(dn):
    if os.path.exists(dn):
        shutil.rmtree(dn)
    os.makedirs(dn)


def measure_map( label_lst,pred_lst ):
    
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    
    for i in range(len(label_lst)):

        if label_lst[i] == 1:     
            if pred_lst[i] == 1:
                TP += 1
            elif pred_lst[i] == 0:
                FN += 1
            else:
                raise
        elif label_lst[i] == 0:         
            if pred_lst[i] == 1:
                FP += 1
            elif pred_lst[i] == 0:
                TN += 1
            else:
                raise
        else:
            raise                
                
                
    return TP,FP,FN,TN


def PRECISION( TP,FP,FN,TN ):    
    
    if TP + FP == 0:
        return 0.0
        # return np.nan
    

    return TP / ( TP + FP )


def RECALL( TP,FP,FN,TN ):

    if TP + FN == 0:
        return 0.0
        # return np.nan    
    
    return TP / ( TP + FN )

def ACCURACY( TP,FP,FN,TN ):

    if TP + FP + TN + FN == 0:
        return 0.0
        # return np.nan    
    
    return (TP+TN) / ( TP + FP + TN + FN )

def F1( TP,FP,FN,TN ):
    
    recall = RECALL( TP,FP,FN,TN )
    precision = PRECISION( TP,FP,FN,TN )
    
    if recall + precision == 0:
        return 0.0
        # return np.nan

    return 2 * recall * precision / ( recall + precision )

if int_occ_mode == "occ":
    
    intensity_level = 2

    def get_label_name( intensity ):
        if intensity == 0:
            return "NOOCC"
        elif intensity == 1:
            return "OCC"
        else:
            raise
            
    measurement_name = "F1"
    
    def get_measurement_score( label_lst, pred_lst ):
        TP,FP,FN,TN = measure_map( label_lst,pred_lst )
        
        return F1( TP,FP,FN,TN )

In [ ]:
def load_img( img_name_0 ):
    with open(img_name_0, 'rb') as f:
        img_0 = Image.open(f)
        img_0 = img_0.convert('RGB')

    img_0 = transforms.ToTensor()(img_0)
    img_0 = transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])(img_0)
    
    return img_0

def get_best_epoch( filename, category, direct, dataset_path_lst ):

    fp = open(filename,"r")
    epoch_score_lst = json.load( fp )
    fp.close()

    pose_lst = list(dataset_path_lst["Valid"].keys())
    best_epoch = 0
    for epoch in range(len(epoch_score_lst["Valid"][pose_lst[0]][category])):

        score_cur_best = np.average([epoch_score_lst["Valid"][pose][category][best_epoch] for pose in pose_lst])
        score = np.average([epoch_score_lst["Valid"][pose][category][epoch] for pose in pose_lst])
        
        if direct == "UPPER":
            if score_cur_best < score:
                best_epoch = epoch
        elif direct == "LOWER":
            if score_cur_best > score:
                best_epoch = epoch
                
                
                
    print("best_epoch",best_epoch)
      
    print("---")
    
    for phase in epoch_score_lst.keys():
        pose_lst = list(dataset_path_lst[phase].keys())
        for c in epoch_score_lst[phase][pose_lst[0]].keys():
            
            score_lst = [epoch_score_lst[phase][pose][c][best_epoch] for pose in pose_lst]
            print(phase,c,"AVERAGE:",np.average(score_lst),"STD:",np.std(score_lst))
            
    return best_epoch

def show_score_lst( filename, dataset_path_lst ):

    fp = open(filename,"r")
    epoch_score_lst = json.load( fp )
    fp.close()
    
    pose_lst = list(dataset_path_lst["Valid"].keys())
    for c in epoch_score_lst["Valid"][pose_lst[0]].keys():
        for phase in epoch_score_lst.keys():
            pose_lst = list(dataset_path_lst[phase].keys())
            en = len(epoch_score_lst[phase][pose_lst[0]][c])
            score_lst = [ np.average([ epoch_score_lst[phase][pose][c][epoch] for pose in pose_lst ]) for epoch in range(en) ]
            
            plt.plot(list(range(len(score_lst))),score_lst,label="%s %s" % (phase,c))

        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
        plt.title(c)
        plt.show()

# Initialize directories

In [ ]:
init_dir(dataset_lst_dirname)
init_dir("./pred_result")
init_dir("./log")
init_dir("./model")
init_dir("./tmp")

# Construct dataset

## Definition of functions

In [ ]:
def get_filename_lst_cache(dataset_path_lst,au_name):
    
    filename_lst_cache_all = []
    for dataset_name,dataset_path_ll in dataset_path_lst.items():
        filename_lst_cache = {}
        occ_int_flag = "int"
        pose_mode = "single_pose"
        pose_prev = None
        
        for intensity in range(6):
            filename_lst_cache[intensity] = {}
            
            for dataset_path in dataset_path_ll:    


                if dataset_path.find( "frames_for_pytorch_procrustes_occ_pose" ) >= 0:

                    occ_int_flag = "occ"

                    if intensity > 1:
                        continue

                    dirname = dataset_path + "/" + au_name + "/" + ( "NOOCC" if intensity == 0 else "OCC" )

                elif  dataset_path.find( "frames_for_pytorch_procrustes_int_pose" ) >= 0:

                    dirname = dataset_path + "/" + au_name + "/" + ("INT%d" % intensity)
                else:
                    raise

                for filename in sorted(glob.glob(dirname + "/*.jpg")):
                    subject = os.path.basename(filename).split("_")[2] + "_" + os.path.basename(filename).split("_")[3]
                    pose = int(os.path.basename(filename).split("_")[4])
                    frame_id = int(os.path.splitext(os.path.basename(filename))[0].split("_")[5][len("frame"):])

                    if (not pose_prev is None) and (pose != pose_prev):
                        pose_mode = "multi_pose"
                    pose_prev = pose

                    if not subject in filename_lst_cache[intensity]:
                        filename_lst_cache[intensity][subject] = [[],[],[]]

                    filename_lst_cache[intensity][subject][0].append(filename)
                    filename_lst_cache[intensity][subject][1].append(pose)
                    filename_lst_cache[intensity][subject][2].append(frame_id)
                
        filename_lst_cache_all.append((filename_lst_cache,occ_int_flag,pose_mode,dataset_name))
        
    return filename_lst_cache_all

## Make csv file of dataset

### for training pseudo-intensity model

In [ ]:
for au_name in au_name_lst:    
    for phase in phase_lst:
        for pose in dataset_path_lst[phase].keys():
        
            csv_filename = dataset_lst_dirname + "/AU_INT_PAIR_%s_%s_%s.csv" % (au_name,phase,pose)

            print("---------------------")
            
            print(csv_filename)

            filename_lst_cache_all = get_filename_lst_cache(dataset_path_lst[phase][pose],au_name)

            
            for i in range(len(filename_lst_cache_all)):
                print(filename_lst_cache_all[i][1],filename_lst_cache_all[i][2],filename_lst_cache_all[i][3])

            fp = open(csv_filename,"w")

            counter = 0
            while counter < dataset_num_lst[phase]:

                dataset_id = random.sample(list(range(len(filename_lst_cache_all))),1)[0]
                filename_lst_cache,occ_int_flag,pose_mode,_ = filename_lst_cache_all[dataset_id]

                intensity_pair_lst = []
                for intensity_0 in range(2 if occ_int_flag == "occ" else 6):
                    for intensity_1 in range(2 if occ_int_flag == "occ" else 6):

                        if enable_pair_dataset_equal_test and phase == "Test":
                            pass
                        
                        else:                        
                            if abs(intensity_0 - intensity_1) == 0:
                                continue
                        
                        intensity_pair_lst.append((intensity_0,intensity_1))
                        
                target_intensity_pair = random.sample(intensity_pair_lst,1)[0]               
                
                
                intensity_0 = target_intensity_pair[0]
                intensity_1 = target_intensity_pair[1]


                if intensity_0 < intensity_1:
                    label = 1
                elif intensity_0 > intensity_1:
                    label = 0
                else:
                    if enable_pair_dataset_equal_test and phase == "Test":
                        label = 0
                    else:
                        raise


                subject_lst_0 = sorted(list(filename_lst_cache[intensity_0].keys()))
                subject_lst_1 = sorted(list(filename_lst_cache[intensity_1].keys()))
                                

                if len(subject_lst_0) == 0:
                    continue
                if len(subject_lst_1) == 0:
                    continue

                pair_lst = []
                for s0 in subject_lst_0:
                    for s1 in subject_lst_1:
                        if s0 == s1:
                            pair_lst.append((s0,s1))                          


                if len(pair_lst) == 0:
                    continue


                subject_0,subject_1 = random.sample(pair_lst,1)[0]


                filename_lst_0,pose_lst_0,frame_id_lst_0 = filename_lst_cache[intensity_0][subject_0]
                filename_lst_1,pose_lst_1,frame_id_lst_1 = filename_lst_cache[intensity_1][subject_1]

                if len(filename_lst_0) == 0:
                    continue
                if len(filename_lst_1) == 0:
                    continue

                filename_0 = random.sample(filename_lst_0,1)[0]
                filename_1 = random.sample(filename_lst_1,1)[0]

                fp.write("%s,%s,%d\n" % (os.path.abspath(filename_0),os.path.abspath(filename_1),label))
                counter += 1


            fp.close()        

### for training mapping model

In [ ]:
for au_name in au_name_lst:    
    for phase in phase_lst:
        for pose in intensity_tune_dataset_path_lst[phase].keys():
            csv_filename = dataset_lst_dirname + "/AU_INT_TUNE_%s_%s_%s.csv" % (au_name,phase,pose)

            print(csv_filename)

            fp = open(csv_filename,"w")
            
            for dataset_name in intensity_tune_dataset_path_lst[phase][pose].keys():

                filename_lst = []
                for cdirname in intensity_tune_dataset_path_lst[phase][pose][dataset_name]:
                    for intensity in range(intensity_level):
                        for filename in sorted(glob.glob(cdirname + "/" + au_name + "/" + get_label_name(intensity) + "/*.jpg")):

                            if small_debug_mode:
                                if random.random() > 0.01:
                                    continue

                            fp.write("%s,%d\n" % (os.path.abspath(filename),intensity))
           
            fp.close()

In [ ]:
for au_name in au_name_lst:    
    for phase in phase_lst:
        for pose in intensity_tune_dataset_path_lst[phase].keys():
            csv_filename = dataset_lst_dirname + "/AU_INT_TUNE_MINI_%s_%s_%s.csv" % (au_name,phase,pose)

            print(csv_filename)

            fp = open(csv_filename,"w")
            
            for dataset_name in intensity_tune_dataset_path_lst[phase][pose].keys():

                for cdirname in intensity_tune_dataset_path_lst[phase][pose][dataset_name]:
                    filename_lst = {}
                    
                    for intensity in range(intensity_level):
                        for filename in sorted(glob.glob(cdirname + "/" + au_name + "/" + get_label_name(intensity) + "/*.jpg")):
                            
                            subject_task_pose = os.path.basename(filename).split("_")[2] + "_" + os.path.basename(filename).split("_")[3] + "_" + os.path.basename(filename).split("_")[4]
                            if not subject_task_pose in filename_lst:
                                filename_lst[subject_task_pose] = []
                            filename_lst[subject_task_pose].append((filename,intensity))
                    

                    for subject_task_pose, filename_intensity_lst in filename_lst.items():

                        sampling_num = min(max(int(len(filename_intensity_lst) * intensity_tune_sampling_ratio),intensity_tune_sampling_min),len(filename_intensity_lst))
                        
                        for filename,intensity in random.sample(filename_intensity_lst,sampling_num):
                            fp.write("%s,%d\n" % (os.path.abspath(filename),intensity))
            fp.close()

# Train and Test

## Train pseudo-intensity model

### Define dataloader

In [ ]:
class PairDataset(torch.utils.data.Dataset):

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file,header=None)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name_0 = self.df.iloc[idx,0]
        img_name_1 = self.df.iloc[idx,1]
        label = self.df.iloc[idx,2]
       
        img_0 = load_img( img_name_0 )
        img_1 = load_img( img_name_1 )

        return img_0,img_1,label,img_name_0,img_name_1



In [ ]:
def get_dataloaders_for_pairwise_net( dataset_path_lst, dataset_lst_dirname, au_name_lst, phase_lst, batch_size_bin ):

    dataloaders = {}

    for au_name in au_name_lst:
        dataloaders[au_name] = {}
        for phase in phase_lst:
            dataloaders[au_name][phase] = {}
            for pose in dataset_path_lst[phase].keys():

                csv_filename = dataset_lst_dirname + "/AU_INT_PAIR_%s_%s_%s.csv" % (au_name,phase,pose)

                pair_dataset = PairDataset(csv_filename)
                dataloader = torch.utils.data.DataLoader(pair_dataset, batch_size=batch_size_bin, shuffle=True, num_workers=1)

                dataloaders[au_name][phase][pose] = dataloader
                
    return dataloaders

### Run training process

In [ ]:
class PairNet(nn.Module):
    def __init__(self):
        super(PairNet, self).__init__()
        
        dropout = 0.5

        self.model = models.vgg16(pretrained=True)
        

        for name, param in self.model.named_parameters():
            if ('features.0.' in name or 'features.2.' in name):
                param.requires_grad = False                    


        self.model.classifier[2].p = dropout
        self.model.classifier[5].p = dropout

        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 1)

    def forward(self, img_0, img_1):
        
        feature_0 = self.model.forward(img_0)
        feature_1 = self.model.forward(img_1)
        
        x_0 = torch.add(torch.mul(feature_1,-1),feature_0)
        x_1 = torch.add(torch.mul(feature_0,-1),feature_1)        
        
        x = torch.cat([x_0,x_1],dim=1)
        
        return x
        

    def forward_single(self, img_0):
        
        feature_0 = self.model.forward(img_0)

        return feature_0
    
def my_loss_func(output,label):

    output = output[:,0]
    label = torch.add(torch.mul(label,2),-1)
    m = 1    
    loss_element = torch.clamp(torch.add(torch.mul( output, label ),m), min=0)
    loss = torch.mean(loss_element)
    
    return loss

In [ ]:
def train_pairwise_net( dataloaders, dataset_path_lst, au_name_lst, phase_lst, device, train_size_per_epoch ):

    for au_name in au_name_lst:

        print("=====================")
        print("au_name",au_name)

        pair_net = PairNet()
        pair_net = pair_net.to(device)
        optimizer = optim.Adam(pair_net.parameters(), lr=0.00005)

        epoch_score_lst = { phase: { pose: { category: [] for category in ["ACC","LOSS"] } for pose in dataset_path_lst[phase].keys() } for phase in phase_lst }



        total_train_size = 0
        epoch = 0    

        # score
        running_corrects = 0
        total_input_size = 0
        running_loss = 0

        # Train flag
        pair_net.train()
        pair_net.model.train()


        for img_0_batch, img_1_batch, label_batch, path_0_batch, path_1_batch in dataloaders[au_name]["Train"]["posesall"]:

            # Train
            img_0_batch = img_0_batch.to(device)
            img_1_batch = img_1_batch.to(device)
            label_batch = label_batch.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):

                output_batch = pair_net.forward(img_0_batch,img_1_batch)
                _, pred_batch = torch.max(output_batch, 1)

                loss = my_loss_func(output_batch, label_batch) # criterion(output_batch, label_batch)

                loss.backward()
                optimizer.step()

                running_loss += loss.item() * img_0_batch.size(0)
                running_corrects += torch.sum(pred_batch == label_batch.data)
                total_input_size += img_0_batch.size(0)


            total_train_size += img_0_batch.size(0)

            # Valid, Test
            if total_train_size >= (epoch + 1) * train_size_per_epoch:

                print("---------------------")

                print("phase","Train")
                print("epoch",epoch)
                print("train_size",total_train_size)

                epoch_acc = float((running_corrects.double()  / total_input_size).to('cpu').detach().numpy().copy())
                epoch_loss = running_loss / total_input_size

                print("epoch_acc",epoch_acc)
                print("epoch_loss",epoch_loss)

                epoch_score_lst["Train"]["posesall"]["ACC"].append(epoch_acc)
                epoch_score_lst["Train"]["posesall"]["LOSS"].append(epoch_loss)



                # Eval flag
                pair_net.eval()
                pair_net.model.eval()

                for phase in ["Valid","Test"]:
                    for pose in dataset_path_lst[phase].keys():

                        print("---")

                        print("phase",phase)
                        print("epoch",epoch)
                        print("pose",pose)
                        print("train_size",total_train_size)

                        # save
                        fp_pred_result = open("./pred_result/pair_net_%s_epoch_%d_phase_%s_%s.csv" % (au_name,epoch,phase,pose),"w")
                        fp_pred_result.write("path_0,path_1,label,pred\n")



                        running_corrects = 0
                        total_input_size = 0
                        running_loss = 0

                        pred_lst = []
                        label_lst = []
                        path_0_lst = []
                        path_1_lst = []
                        prev_size = 0

                        for img_0_batch, img_1_batch, label_batch, path_0_batch, path_1_batch in dataloaders[au_name][phase][pose]:

                            img_0_batch = img_0_batch.to(device)
                            img_1_batch = img_1_batch.to(device)
                            label_batch = label_batch.to(device)

                            optimizer.zero_grad()

                            with torch.set_grad_enabled(False):

                                output_batch = pair_net.forward(img_0_batch,img_1_batch)
                                _, pred_batch = torch.max(output_batch, 1)

                                loss = my_loss_func(output_batch, label_batch) # criterion(output_batch, label_batch)

                            running_loss += loss.item() * img_0_batch.size(0)
                            running_corrects += torch.sum(pred_batch == label_batch.data)
                            total_input_size += img_0_batch.size(0)

                            pred_lst += list(pred_batch.to('cpu').detach().numpy().copy())
                            label_lst += list(label_batch.to('cpu').detach().numpy().copy())
                            path_0_lst += list(path_0_batch)                       
                            path_1_lst += list(path_1_batch)  


                            # save
                            for i in range(prev_size,len(path_0_lst)):
                                fp_pred_result.write("%s,%s,%d,%d\n" % (path_0_lst[i],path_1_lst[i],int(label_lst[i]),int(pred_lst[i])) )                
                            prev_size = len(path_0_lst)


                        epoch_acc = float((running_corrects.double()  / total_input_size).to('cpu').detach().numpy().copy())
                        epoch_loss = running_loss / total_input_size

                        print("epoch_acc",epoch_acc)
                        print("epoch_loss",epoch_loss)

                        fp_pred_result.close()

                        epoch_score_lst[phase][pose]["ACC"].append(epoch_acc)
                        epoch_score_lst[phase][pose]["LOSS"].append(epoch_loss)     


                # save
                fp = open("./log/pair_net_epoch_score_lst_%s_%s.json" % (au_name, "posesall") ,"w")
                json.dump( epoch_score_lst, fp )
                fp.close()



                # score
                running_corrects = 0
                total_input_size = 0
                running_loss = 0


                # Train flag
                pair_net.train()
                pair_net.model.train()

                # save
                torch.save(pair_net.state_dict(), "./model/pair_net_%s_epoch_%d_%s.pt" % (au_name, epoch, "posesall" ) )

                epoch += 1
                
        del pair_net


In [ ]:
def get_pairwise_net_best_epoch_lst( dataset_path_lst, au_name_lst ):

    pair_net_best_epoch_lst = {}

    for au_name in au_name_lst:
        print("=============")
        print(au_name)

        filename = "./log/pair_net_epoch_score_lst_%s_%s.json" % (au_name,"posesall")

        pair_net_best_epoch_lst[au_name] = get_best_epoch( filename, "LOSS", "LOWER", dataset_path_lst )
        show_score_lst( filename, dataset_path_lst )

    return pair_net_best_epoch_lst


### Free memory

In [ ]:
def memory_free_pairwise_net_dataloaders( dataloaders ):
    for au_name in dataloaders.keys():
        for phase in dataloaders[au_name].keys():
            for pose in list(dataloaders[au_name][phase].keys()):
                del dataloaders[au_name][phase][pose]

## Predict pseudo-intensity

### Define dataloader

In [ ]:
# Dataset
class IntensityTuneDataset(torch.utils.data.Dataset):

    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file,header=None)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name_0 = self.df.iloc[idx,0]
        label = self.df.iloc[idx,1]
       
        img_0 = load_img( img_name_0 )

        return img_0,label,img_name_0


def get_dataloaders_for_intensity_tune( intensity_tune_dataset_path_lst, au_name_lst, phase_lst, batch_size_int, is_mini ):

    dataloaders_intensity_tune = {}

    for au_name in au_name_lst:
        dataloaders_intensity_tune[au_name] = {}
        for phase in phase_lst:
            dataloaders_intensity_tune[au_name][phase] = {}
            for pose in intensity_tune_dataset_path_lst[phase].keys():

                csv_filename = dataset_lst_dirname + "/AU_INT_TUNE%s_%s_%s_%s.csv" % ( ("_MINI" if is_mini else ""), au_name,phase,pose)

                intensity_tune_dataset = IntensityTuneDataset(csv_filename)
                dataloader = torch.utils.data.DataLoader(intensity_tune_dataset, batch_size=batch_size_int, shuffle=True, num_workers=1)

                dataloaders_intensity_tune[au_name][phase][pose] = dataloader
    return dataloaders_intensity_tune

### Run predicting process

In [ ]:
def predict_pairwise_net( pair_net_best_filename_lst, intensity_tune_dataset_path_lst, dataloaders_intensity_tune, au_name_lst, phase_lst, device, is_mini ):

    for au_name in au_name_lst:

        pair_net = PairNet()
        pair_net.load_state_dict(torch.load(pair_net_best_filename_lst[au_name]))
        pair_net.eval()
        pair_net.to(device)

        for phase in phase_lst:

            print("------------")
            print(au_name,phase)

            for pose in intensity_tune_dataset_path_lst[phase].keys():
                print("phase",phase,"pose",pose)

                pred_lst_pose = []
                label_lst_pose = []
                path_lst_pose = []

                for img_0_batch, label_batch, path_batch in dataloaders_intensity_tune[au_name][phase][pose]:

                    img_0_batch = img_0_batch.to(device)
                    with torch.set_grad_enabled(False):

                        output_batch = pair_net.forward_single(img_0_batch)

                    pred_batch = output_batch.to('cpu').detach().numpy().copy()

                    pred_lst_pose += list(pred_batch)
                    label_lst_pose += list(label_batch)
                    path_lst_pose += list(path_batch)


                fp_pred_result_raw = open("./pred_result/pred_result__%s_procrustes-intensity-%s_%s_psedo%s.csv" % (au_name,pose,phase, "_mini" if is_mini else "" ),"w")
                fp_pred_result_raw.write("path,label,pred\n")

                for i in range(len(path_lst_pose)):
                    fp_pred_result_raw.write("%s,%d,%f\n"%(path_lst_pose[i],label_lst_pose[i],pred_lst_pose[i]))

                fp_pred_result_raw.close()            


        # free menory
        del pair_net

### Free memory

In [ ]:
def memeory_free_dataloaders_intensity_tune( dataloaders_intensity_tune ):

    for au_name in dataloaders_intensity_tune.keys():
        for phase in dataloaders_intensity_tune[au_name].keys():
            for pose in list(dataloaders_intensity_tune[au_name][phase].keys()):
                del dataloaders_intensity_tune[au_name][phase][pose]

# torch.cuda.empty_cache()

## Train mapping model

### Make csv file of dataset

In [ ]:
def make_dataset_for_intensity_net( intensity_tune_dataset_path_lst, intensity_conv_dataset_num_lst, dataset_lst_dirname, au_name_lst, phase_lst, phase_full_lst, is_mini ):

    for au_name in au_name_lst:    
        for phase in phase_lst + phase_full_lst:
            for pose in intensity_tune_dataset_path_lst[phase].keys():

                phase_tmp = phase
                if phase_tmp == "TrainFull":
                    phase_tmp = "Train"
                if phase_tmp == "ValidFull":
                    phase_tmp = "Valid"
                if phase_tmp == "TestFull":
                    phase_tmp = "Test"

                csv_filename = "./pred_result/pred_result__%s_procrustes-intensity-%s_%s_psedo%s.csv" % (au_name,pose,phase_tmp,"_mini" if is_mini else "")

                dataset_lst = { dataset_name:[os.path.abspath(dataset_path) for dataset_path in _dataset_path_lst] for dataset_name, _dataset_path_lst in intensity_tune_dataset_path_lst[phase][pose].items()}

                dataset_name_lst = list(dataset_lst.keys())

                path_lst = { dataset_name: [] for dataset_name in dataset_lst.keys() }
                label_lst = { dataset_name: [] for dataset_name in dataset_lst.keys() }
                pred_lst = { dataset_name: [] for dataset_name in dataset_lst.keys() }
                subject_lst = { dataset_name: [] for dataset_name in dataset_lst.keys() }

                for line_id,line in enumerate(open(csv_filename)):
                    if line_id == 0:
                        continue

                    path,label,pred = line.rstrip().split(",")
                    label = int(label)
                    pred = float(pred)
                    subject = os.path.basename(path).split("_")[2]

                    dataset_name = None
                    for _dataset_name, _dataset_path_lst in dataset_lst.items():
                        for dataset_path in _dataset_path_lst:
                            if path.find(dataset_path) == 0:
                                dataset_name = _dataset_name
                    if dataset_name is None:
                        raise

                    path_lst[dataset_name].append(path)
                    label_lst[dataset_name].append(label)
                    pred_lst[dataset_name].append(pred)
                    subject_lst[dataset_name].append(subject)


                output_csv_filename = dataset_lst_dirname + "/AU_INT_CONV_%s_%s_%s.csv" % (au_name,phase,pose)

                if phase == "Train":

                    filename_lst_cache_lst = {}

                    for dataset_name in dataset_lst.keys():

                        filename_lst_cache = {}

                        for i in range(len(path_lst[dataset_name])):

                            subject = subject_lst[dataset_name][i]
                            label = label_lst[dataset_name][i]

                            if not label in filename_lst_cache:
                                filename_lst_cache[label] = {}

                            if not subject in filename_lst_cache[label]:
                                filename_lst_cache[label][subject] = []

                            filename_lst_cache[label][subject].append( (path_lst[dataset_name][i], pred_lst[dataset_name][i]) )

                        filename_lst_cache_lst[dataset_name] = filename_lst_cache


                    fp = open(output_csv_filename,"w")

                    counter = 0
                    while counter < intensity_conv_dataset_num_lst[phase]:

                        dataset_name = random.sample(dataset_name_lst,1)[0]
                        filename_lst_cache = filename_lst_cache_lst[dataset_name]
                        if len(list(filename_lst_cache.keys())) == 0:
                            continue

                        intensity = random.sample(list(filename_lst_cache.keys()),1)[0]                    

                        subject = random.sample(sorted(list(filename_lst_cache[intensity].keys())),1)[0]

                        path, pred = random.sample(filename_lst_cache[intensity][subject],1)[0]

                        label = intensity

                        fp.write("%s,%d,%f\n" % (path,label,pred))

                        counter += 1


                    fp.close()


                else:            
                    fp = open(output_csv_filename,"w")
                    print(csv_filename,len(path_lst))

                    for dataset_name in dataset_name_lst:
                        n = intensity_conv_dataset_num_lst[phase] if intensity_conv_dataset_num_lst[phase] >= 0 else len(path_lst[dataset_name])
                        n = min(len(path_lst[dataset_name]),n)
                        
                        print(au_name,phase,pose,dataset_name,"sampling:",n)
                        
                        for i in random.sample(list(range(len(path_lst[dataset_name]))),n):
                            fp.write("%s,%d,%f\n" % (path_lst[dataset_name][i],label_lst[dataset_name][i],pred_lst[dataset_name][i]))

                    fp.close()                



### Define dataloader

In [ ]:
# Dataset
class IntensityConvDataset(torch.utils.data.Dataset):

    def __init__(self, csv_filename_all, csv_filename_target,pred_min,pred_max):
        
        self.subject_feature_lst = {}
        self.pred_min = pred_min
        self.pred_max = pred_max
        
        for line_id,line in enumerate(open(csv_filename_all)):

            if line_id == 0:
                continue

            path,label,pred = line.rstrip().split(",")

            subject = os.path.basename(path).split("_")[2]
            task = os.path.basename(path).split("_")[3]
            pose = os.path.basename(path).split("_")[4]            

            label = int(label)
            pred = float(pred)

            
            if not subject in self.subject_feature_lst:
                self.subject_feature_lst[subject] = {}
            if not pose in self.subject_feature_lst[subject]:
                self.subject_feature_lst[subject][pose] = {}
            if not task in self.subject_feature_lst[subject][pose]:
                self.subject_feature_lst[subject][pose][task] = { "pred_lst":[], "basename_lst":[], "path_lst":[], "frame_id_lst":[] }
                
            self.subject_feature_lst[subject][pose][task]["pred_lst"].append(pred)
            self.subject_feature_lst[subject][pose][task]["path_lst"].append(path)
            
            basename = os.path.splitext(os.path.basename(path))[0]
            self.subject_feature_lst[subject][pose][task]["basename_lst"].append(basename)
            frame_id = int(basename.split("_")[5][len("frame"):])
            self.subject_feature_lst[subject][pose][task]["frame_id_lst"].append(frame_id)
            
            

        for subject in self.subject_feature_lst.keys():
            for pose in self.subject_feature_lst[subject].keys():
                for task in self.subject_feature_lst[subject][pose].keys():
                    
                    frame_id_lst_with_i = []
                    
                    for i in range(len(self.subject_feature_lst[subject][pose][task]["frame_id_lst"])):
                        frame_id_lst_with_i.append((self.subject_feature_lst[subject][pose][task]["frame_id_lst"][i],i))
                    
                    frame_id_lst_with_i.sort()
                    
                    for key in self.subject_feature_lst[subject][pose][task].keys():
                        self.subject_feature_lst[subject][pose][task][key] = [ self.subject_feature_lst[subject][pose][task][key][i] for frame_id,i in frame_id_lst_with_i ]
                    


        pred_lst_len = []
        for subject in self.subject_feature_lst.keys():
            for pose in self.subject_feature_lst[subject].keys():
                for task in self.subject_feature_lst[subject][pose].keys():
                    pred_lst_len.append(len(self.subject_feature_lst[subject][pose][task]["pred_lst"]))


        pred_lst_max_len = max(pred_lst_len)
        print("min:",min(pred_lst_len),"max:",max(pred_lst_len),"mean:",np.mean(pred_lst_len),"std:",np.std(pred_lst_len))

        frame_id_all_lst = []
        for subject in self.subject_feature_lst.keys():
            for pose in self.subject_feature_lst[subject].keys():
                for task in self.subject_feature_lst[subject][pose].keys():
                    frame_id_all_lst += self.subject_feature_lst[subject][pose][task]["frame_id_lst"]
    

        frame_id_min = min(frame_id_all_lst)
        frame_id_max = max(frame_id_all_lst)
        
        for subject in self.subject_feature_lst.keys():
            for pose in self.subject_feature_lst[subject].keys():
                for task in self.subject_feature_lst[subject][pose].keys():
                    self.subject_feature_lst[subject][pose][task]["pred_lst_padding"] = []
                    mean_frame_width = 10
                    value = np.mean(self.subject_feature_lst[subject][pose][task]["pred_lst"][-mean_frame_width:])
                    self.subject_feature_lst[subject][pose][task]["last_padding_value"] = value
                    
                    target_frame_id_id = 0
                    br = False
                    for frame_id in range(frame_id_min,frame_id_max+1):
                        
                        if br:
                            break
                        
                        while True:                           
                            
                            
                            if target_frame_id_id >= len(self.subject_feature_lst[subject][pose][task]["frame_id_lst"]):
                                br = True
                                
                                break
                        
                            target_frame_id = self.subject_feature_lst[subject][pose][task]["frame_id_lst"][target_frame_id_id]

 
                            if frame_id == target_frame_id:
                                self.subject_feature_lst[subject][pose][task]["pred_lst_padding"].append(self.subject_feature_lst[subject][pose][task]["pred_lst"][target_frame_id_id])
                                break
                            elif frame_id < target_frame_id:

                                if len(self.subject_feature_lst[subject][pose][task]["pred_lst_padding"]) == 0:
                                    value = np.mean(self.subject_feature_lst[subject][pose][task]["pred_lst"][0:mean_frame_width])
                                else:
                                    value = np.mean(self.subject_feature_lst[subject][pose][task]["pred_lst_padding"][-mean_frame_width:])

                                self.subject_feature_lst[subject][pose][task]["pred_lst_padding"].append(value)
                    
                                break
                            else:
                                target_frame_id_id += 1
                                

        self.data_lst = []           
        for line_id,line in enumerate(open(csv_filename_target)):          
            
            path,label,pred = line.rstrip().split(",")

            subject = os.path.basename(path).split("_")[2]
            task = os.path.basename(path).split("_")[3]
            pose = os.path.basename(path).split("_")[4]            

            label = int(label)
            pred = float(pred)
            
            self.data_lst.append( (path,subject,pose,task,pred,label) )


        for subject in self.subject_feature_lst.keys():
            for pose in self.subject_feature_lst[subject].keys():
                for task in self.subject_feature_lst[subject][pose].keys():
                    
                    
                    if len(self.subject_feature_lst[subject][pose][task]["pred_lst"]) <= 1:
                        print("[WARN] no frame",len(self.subject_feature_lst[subject][pose][task]["pred_lst"]),self.subject_feature_lst[subject][pose][task]["pred_lst"])
                        self.subject_feature_lst[subject][pose][task]["hist"] = [0.0]*24
                        self.subject_feature_lst[subject][pose][task]["percentile"] = [0.0]*11
                        self.subject_feature_lst[subject][pose][task]["path_min"] = ""                        
                        
                        continue
                        
                    
            
                    hist_n = 6*4
                    
                    kde_model = gaussian_kde([ pred + (random.random()*2-1)*1.0e-10 for pred in self.subject_feature_lst[subject][pose][task]["pred_lst"]])
                    kde_x = np.linspace(pred_min,pred_max,num=hist_n*10)
                    kde_y = kde_model(kde_x)       
                    

                    kde_x_q = np.linspace(pred_min,pred_max,num=hist_n)
                    kde_y_q = [ np.mean([ kde_y[i*10+j] for j in range(10)]) for i in range(hist_n) ]

                    hist = list(kde_y_q)
                    hist_total = sum(hist)
                    if hist_total > 0:
                        hist = [ a/hist_total for a in hist ]

                    self.subject_feature_lst[subject][pose][task]["hist"] = hist

                    
                    # percentile
                    self.subject_feature_lst[subject][pose][task]["percentile"] = []
                    
                    pred_lst_sorted = sorted(self.subject_feature_lst[subject][pose][task]["pred_lst"])
                    
                    for p in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]: # 11
                        self.subject_feature_lst[subject][pose][task]["percentile"].append((pred_lst_sorted[int((len(pred_lst_sorted)-1)*p)]-pred_min)/(pred_max-pred_min)*2-1)
                        
    
        

                    

    def __len__(self):
        return len(self.data_lst)

    def __getitem__(self, idx):
        path,subject,pose,task,psedo_pred,label = self.data_lst[idx]

        
        feature = []
        if "hist" in intensity_net_feature_lst:
            feature += list(self.subject_feature_lst[subject][pose][task]["hist"])
        if "percentile" in intensity_net_feature_lst:
            feature += list(self.subject_feature_lst[subject][pose][task]["percentile"])
       
        return torch.tensor(feature), \
               torch.tensor([(psedo_pred-self.pred_min)/(self.pred_max-self.pred_min)*2-1]), \
               torch.tensor(label), \
               path, \
               None, \
               None, \
               None
               

                

In [ ]:
# pred_min,pred_max
def get_pred_min_pred_max( intensity_tune_dataset_path_lst, au_name_lst, is_mini ):
    
    intensity_net_param_lst = {}

    for au_name in au_name_lst:
        
        pred_min = 100000
        pred_max = -100000
    
        for phase in ["Train"]:
            for pose in intensity_tune_dataset_path_lst[phase].keys():

                csv_filename_all ="./pred_result/pred_result__%s_procrustes-intensity-%s_%s_psedo%s.csv" % (au_name,pose,phase,"_mini" if is_mini else "")

                for line_id,line in enumerate(open(csv_filename_all)):

                    if line_id == 0:
                        continue

                    path,label,pred = line.rstrip().split(",")
                    pred = float(pred)

                    if pred_min > pred:
                        pred_min = pred
                    if pred_max < pred:
                        pred_max = pred
            
        print(pred_min,pred_max)
        intensity_net_param_lst[au_name] = { "pred_min":pred_min, "pred_max":pred_max }
    
    return intensity_net_param_lst

In [ ]:
def dataloader_collate_fn(batch):
    feature, psedo_pred, label, path, _, _, _ = list(zip(*batch))
    feature = torch.stack(feature)
    psedo_pred = torch.stack(psedo_pred)
    label = torch.stack(label) 
    
    return feature,psedo_pred,label,path,None,None

In [ ]:
def get_dataloaders_for_intensity_net( intensity_tune_dataset_path_lst, au_name_lst, phase_lst, phase_full_lst, intensity_net_param_lst, batch_size_int, is_mini ):

    dataloaders_intensity_conv = {}

    for au_name in au_name_lst:
        pred_min = intensity_net_param_lst[au_name]["pred_min"]
        pred_max = intensity_net_param_lst[au_name]["pred_max"]
        dataloaders_intensity_conv[au_name] = {}
        for phase in phase_lst + phase_full_lst:
            dataloaders_intensity_conv[au_name][phase] = {}
            for pose in intensity_tune_dataset_path_lst[phase].keys():

                print("=========")
                print(phase,pose)

                phase_tmp = phase
                if phase_tmp == "TrainFull":
                    phase_tmp = "Train"
                if phase_tmp == "ValidFull":
                    phase_tmp = "Valid"
                if phase_tmp == "TestFull":
                    phase_tmp = "Test"

                csv_filename_all ="./pred_result/pred_result__%s_procrustes-intensity-%s_%s_psedo%s.csv" % (au_name,pose, phase_tmp, "_mini" if is_mini else "")
                csv_filename_target =dataset_lst_dirname + "/AU_INT_CONV_%s_%s_%s.csv" % (au_name,phase,pose)

                intensity_conv_dataset = IntensityConvDataset(csv_filename_all,csv_filename_target,pred_min,pred_max)
                dataloader = torch.utils.data.DataLoader(intensity_conv_dataset, batch_size=batch_size_int, shuffle=True, num_workers=1,collate_fn=dataloader_collate_fn)

                dataloaders_intensity_conv[au_name][phase][pose] = dataloader

    return dataloaders_intensity_conv

### Run training process

In [ ]:
class IntensityNet(nn.Module):
    def __init__(self):
        super(IntensityNet, self).__init__()

        classifier_in_dim = 1
        
        if "hist" in intensity_net_feature_lst:
            classifier_in_dim += 6*4
        if "percentile" in intensity_net_feature_lst:
            classifier_in_dim += 11

        
        self.img_features = models.vgg16(pretrained=True).features      
        for name, param in self.img_features.named_parameters():
            if name.find("0.") == 0 or name.find("2.") == 0:
                print("freeze:",name)
                param.requires_grad = False


        self.layers_classifier = []
        self.layers_classifier.append(nn.Linear(in_features=classifier_in_dim, out_features=4096, bias=True))      
        self.layers_classifier.append(nn.ReLU(inplace=True))
        self.layers_classifier.append(nn.Dropout(p=0.5, inplace=False))
        self.layers_classifier.append(nn.Linear(in_features=4096, out_features=4096, bias=True))
        self.layers_classifier.append(nn.ReLU(inplace=True))
        self.layers_classifier.append(nn.Dropout(p=0.5, inplace=False))        
        self.layers_classifier.append(nn.Linear(in_features=4096, out_features=intensity_level, bias=True))
        self.layers_classifier = nn.ModuleList(self.layers_classifier)
        
    def forward(self, x, feature, _, __):


        feature = feature.float()

              
        x = torch.cat([x,feature],dim=1)

        for i in range(len(self.layers_classifier)):
            x = self.layers_classifier[i](x)
        
        return x


In [ ]:
def train_intensity_net( intensity_tune_dataset_path_lst, dataloaders_intensity_conv, au_name_lst, phase_lst, device, epoch_num ):

    for au_name in au_name_lst:

        intensity_net = IntensityNet()
        intensity_net.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(intensity_net.parameters(), lr=0.00005)

        epoch_score_lst = { phase:{ pose:{ measurement_name:[], "LOSS":[] } for pose in intensity_tune_dataset_path_lst[phase].keys() } for phase in phase_lst }

        for epoch in range(epoch_num):
            print("================")
            print("epoch",epoch)

            for phase in phase_lst:

                if phase == "Train":
                    intensity_net.train()
                else:
                    intensity_net.eval()         

                print("------------")
                print(au_name,phase)

                pred_lst = []
                label_lst = []
                path_lst = []

                icc_lst = []

                for pose in intensity_tune_dataset_path_lst[phase].keys():
                    print("phase",phase,"pose",pose)

                    pred_lst_pose = []
                    psedo_pred_lst_pose = []
                    label_lst_pose = []
                    path_lst_pose = []

                    running_loss = 0
                    running_corrects = 0
                    total_input_size = 0


                    for feature_batch, psedo_pred_batch, label_batch, path_batch, _, _ in dataloaders_intensity_conv[au_name][phase][pose]:

                        feature_batch = feature_batch.to(device)
                        psedo_pred_batch = psedo_pred_batch.to(device)
                        label_batch_org = label_batch
                        label_batch = label_batch.to(device)


                        optimizer.zero_grad()

                        with torch.set_grad_enabled(phase == 'Train'):

                            output_batch = intensity_net.forward(psedo_pred_batch,feature_batch,_,_)
                            _, pred_batch = torch.max(output_batch, 1)

                            loss = criterion(output_batch, label_batch)

                            if phase == 'Train':
                                loss.backward()
                                optimizer.step()

                            running_loss += loss.item() * feature_batch.size(0)
                            running_corrects += torch.sum(pred_batch == label_batch.data)
                            total_input_size += feature_batch.size(0)

                        pred_lst_pose += list(pred_batch.to('cpu').detach().numpy().copy())
                        psedo_pred_lst_pose += list(psedo_pred_batch.to('cpu').detach().numpy().copy())
                        label_lst_pose += list(label_batch_org.numpy().copy())
                        path_lst_pose += list(path_batch)


                    cur_icc = get_measurement_score(label_lst_pose,pred_lst_pose)
                    print(measurement_name,cur_icc)
                    icc_lst.append(cur_icc)

                    loss_pose = running_loss / total_input_size

                    pred_lst += pred_lst_pose
                    label_lst += label_lst_pose
                    path_lst += path_lst_pose

                    epoch_score_lst[phase][pose][measurement_name].append(cur_icc)
                    epoch_score_lst[phase][pose]["LOSS"].append(loss_pose)




                print(measurement_name,"(MEAN)",np.average(icc_lst))




                # save
                fp = open("./log/intensity_net_epoch_score_lst_%s_%s.json" % (au_name, "posesall") ,"w")
                json.dump( epoch_score_lst, fp )
                fp.close()


                # save
                if phase == "Train":
                    torch.save(intensity_net.state_dict(), "./model/intensity_net_%s_epoch_%d_%s.pt" % (au_name, epoch, "posesall" ) )

        del intensity_net

### Get best_epoch

In [ ]:
def get_intensity_net_best_epoch_lst( dataset_path_lst, au_name_lst ):
    
    intensity_net_best_epoch_lst = {}

    for au_name in au_name_lst:
        print("=============")
        print(au_name)

        filename = "./log/intensity_net_epoch_score_lst_%s_%s.json" % (au_name,"posesall")

        intensity_net_best_epoch_lst[au_name] = get_best_epoch( filename, "LOSS", "LOWER", dataset_path_lst )
        show_score_lst( filename, dataset_path_lst )
    
    return intensity_net_best_epoch_lst

### Predict intensity of all data

In [ ]:
def predict_all_testset( dataloaders_intensity_conv, intensity_tune_dataset_path_lst, intensity_net_best_filename_lst, au_name_lst, device, phase_full_lst ):

    softmax = nn.Softmax(dim=1)

    for au_name in au_name_lst:

        intensity_net = IntensityNet()
        intensity_net.load_state_dict(torch.load(intensity_net_best_filename_lst[au_name]))
        intensity_net.eval()
        intensity_net.to(device)

        for phase in phase_full_lst:

            print("------------")
            print(au_name,phase)

            pred_lst = []
            label_lst = []
            path_lst = []

            icc_lst = []

            for pose in intensity_tune_dataset_path_lst[phase].keys():
                print("phase",phase,"pose",pose)

                pred_lst_pose = []
                psedo_pred_lst_pose = []
                label_lst_pose = []
                path_lst_pose = []
                pred_prob_lst_pose = []

                running_corrects = 0
                total_input_size = 0


                for feature_batch, psedo_pred_batch, label_batch, path_batch, _, _ in dataloaders_intensity_conv[au_name][phase][pose]:

                    feature_batch = feature_batch.to(device)
                    psedo_pred_batch = psedo_pred_batch.to(device)
                    label_batch_org = label_batch
                    label_batch = label_batch.to(device)


                    with torch.set_grad_enabled(False):

                        output_batch = intensity_net.forward(psedo_pred_batch,feature_batch,_,_)
                        _, pred_batch = torch.max(output_batch, 1)
                        pred_prob_batch = softmax(output_batch)

                        running_corrects += torch.sum(pred_batch == label_batch.data)
                        total_input_size += feature_batch.size(0)

                    pred_lst_pose += list(pred_batch.to('cpu').detach().numpy().copy())
                    psedo_pred_lst_pose += list(psedo_pred_batch.to('cpu').detach().numpy().copy())
                    label_lst_pose += list(label_batch_org.numpy().copy())
                    path_lst_pose += list(path_batch)                
                    pred_prob_lst_pose += list(pred_prob_batch.to('cpu').detach().numpy().copy())


                # eval
                cur_icc = get_measurement_score(label_lst_pose,pred_lst_pose)
                print(measurement_name,cur_icc)
                print("len",len(label_lst_pose))
                icc_lst.append(cur_icc)

                pred_lst += pred_lst_pose
                label_lst += label_lst_pose
                path_lst += path_lst_pose


                # save
                fp_pred_result = open("./pred_result/pred_result__%s_procrustes-intensity-%s_%s.csv" % (au_name,pose,phase),"w")
                fp_pred_result.write("path,label,pred\n")

                for i in range(len(path_lst_pose)):
                    fp_pred_result.write("%s,%d,%d\n"%(path_lst_pose[i],label_lst_pose[i],pred_lst_pose[i]))

                fp_pred_result.close()


                # save
                fp_pred_result_raw = open("./pred_result/pred_result__%s_procrustes-intensity-%s_%s_psedo_norm.csv" % (au_name,pose,phase),"w")
                fp_pred_result_raw.write("path,label,pred\n")

                for i in range(len(path_lst_pose)):
                    fp_pred_result_raw.write("%s,%d,%f\n"%(path_lst_pose[i],label_lst_pose[i],psedo_pred_lst_pose[i]))
                fp_pred_result_raw.close()

                # save
                fp_pred_result_prob = open("./pred_result/pred_result__%s_procrustes-intensity-%s_%s_prob.csv" % (au_name,pose,phase),"w")
                fp_pred_result_prob.write("path,label,%s\n" % ",".join([ "prob_%d" % intensity for intensity in range(intensity_level) ]) )

                for i in range(len(path_lst_pose)):
                    fp_pred_result_prob.write("%s,%d"%(path_lst_pose[i],label_lst_pose[i]))
                    for intensity in range(intensity_level):
                        fp_pred_result_prob.write(",%.10f"%pred_prob_lst_pose[i][intensity])
                    fp_pred_result_prob.write("\n")
                fp_pred_result_prob.close()



            # eval
            print(measurement_name,"(MEAN)",np.average(icc_lst))
    
    
    
    


### Free memory

In [ ]:
def memory_free_intensity_net_dataloaders( dataloaders_intensity_conv ):
    for au_name in dataloaders_intensity_conv.keys():
        for phase in dataloaders_intensity_conv[au_name].keys():
            for pose in list(dataloaders_intensity_conv[au_name][phase].keys()):
                del dataloaders_intensity_conv[au_name][phase][pose]

# Run

In [ ]:
def write_summary_log( fp_summary, pair_net_epoch_score_lst, au_name, phase_lst, network_name, best_epoch ):
    
    best_validation_score = None
    
    for phase in phase_lst:
        measure_lst = list(pair_net_epoch_score_lst[phase].values())[0].keys()
        for measure in measure_lst:
            epoch_lst = range(len(list(list(pair_net_epoch_score_lst[phase].values())[0].values())[0]))
            for epoch in epoch_lst:
                pose_lst = list(pair_net_epoch_score_lst[phase].keys())
                
                best_epoch_flag = 1 if best_epoch == epoch else 0
                
                score_all_pose_lst = []
                for pose in pose_lst:                    
                    score = pair_net_epoch_score_lst[phase][pose][measure][epoch]
                    score_all_pose_lst.append(score)
                    
                    fp_summary.write("%s,%d,%s,%s,%s,%d,%d,%s,%f\n"%(au_name,trial,network_name,phase,pose,epoch,best_epoch_flag,measure,score))

                fp_summary.write("%s,%d,%s,%s,%s,%d,%d,%s,%f\n"%(au_name,trial,network_name,phase,"pose-mean",epoch,best_epoch_flag,measure,np.mean(score_all_pose_lst)))
                
                if phase == "Valid" and measure == "LOSS" and epoch == best_epoch:
                    best_validation_score = np.mean(score_all_pose_lst)

    fp_summary.flush()
    
    return best_validation_score

In [ ]:
best_validation_loss_lst = { au_name:[] for au_name in au_name_lst }

fp_summary = open("./log/summary.csv","w")
fp_summary.write("au_name,trial,network_name,phase,pose,epoch,best_epoch_flag,measure,score\n")

for trial in range(trial_num):
    
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    print("trial:",trial)

    # train pseudo-intensity model
    dataloaders = get_dataloaders_for_pairwise_net( dataset_path_lst, dataset_lst_dirname, au_name_lst, phase_lst, batch_size_bin )
    train_pairwise_net( dataloaders, dataset_path_lst, au_name_lst, phase_lst, device, train_size_per_epoch )
    pairwise_net_best_epoch_lst = get_pairwise_net_best_epoch_lst( dataset_path_lst, au_name_lst )
    memory_free_pairwise_net_dataloaders( dataloaders )
    
    # copy model of best_epoch
    pair_net_best_filename_lst = {}
    for au_name in au_name_lst:
        shutil.copyfile( "./model/pair_net_%s_epoch_%d_%s.pt" % (au_name, pairwise_net_best_epoch_lst[au_name], "posesall" ),
                         "./model/pair_net_%s_epoch_best_%s_trial_%d.pt" % (au_name, "posesall", trial ))
        pair_net_best_filename_lst[au_name] = "./model/pair_net_%s_epoch_best_%s_trial_%d.pt" % (au_name, "posesall", trial )
    
    # predict pseudo-intensity
    dataloaders_intensity_tune = get_dataloaders_for_intensity_tune( intensity_tune_dataset_path_lst, au_name_lst, phase_lst, batch_size_int, True )
    predict_pairwise_net( pair_net_best_filename_lst, intensity_tune_dataset_path_lst, dataloaders_intensity_tune, au_name_lst, phase_lst, device, True )
    memeory_free_dataloaders_intensity_tune( dataloaders_intensity_tune )
    

    # train mapping model
    make_dataset_for_intensity_net( intensity_tune_dataset_path_lst, intensity_conv_dataset_num_lst, dataset_lst_dirname, au_name_lst, phase_lst, phase_full_lst, True )
    intensity_net_param_lst = get_pred_min_pred_max( intensity_tune_dataset_path_lst, au_name_lst, True )
    dataloaders_intensity_conv = get_dataloaders_for_intensity_net( intensity_tune_dataset_path_lst, au_name_lst, phase_lst, phase_full_lst, intensity_net_param_lst, batch_size_int, True )
    train_intensity_net( intensity_tune_dataset_path_lst, dataloaders_intensity_conv, au_name_lst, phase_lst, device, epoch_num )
    intensity_net_best_epoch_lst = get_intensity_net_best_epoch_lst( dataset_path_lst, au_name_lst )
    memory_free_intensity_net_dataloaders( dataloaders_intensity_conv )
    
    # copy model of best_epoch
    for au_name in au_name_lst:
        shutil.copyfile( "./model/intensity_net_%s_epoch_%d_%s.pt" % (au_name, intensity_net_best_epoch_lst[au_name],"posesall"),
                         "./model/intensity_net_%s_epoch_best_%s_trial_%d.pt" % (au_name, "posesall", trial ))

    # copy parameters
    fp = open("./model/intensity_net_param_lst_trial_%d.json" % trial,"w")
    json.dump( intensity_net_param_lst, fp )
    fp.close()
        
    # save result
    for au_name in au_name_lst:

        network_name = "pairwise"
        best_epoch = pairwise_net_best_epoch_lst[au_name]
        fp = open("./log/pair_net_epoch_score_lst_%s_posesall.json" % au_name)
        pair_net_epoch_score_lst = json.load( fp )
        fp.close()
        write_summary_log( fp_summary, pair_net_epoch_score_lst, au_name, phase_lst, network_name, best_epoch )


        network_name = "intensity"
        best_epoch = intensity_net_best_epoch_lst[au_name]
        fp = open("./log/intensity_net_epoch_score_lst_%s_posesall.json" % au_name)
        intensity_net_epoch_score_lst = json.load( fp )
        fp.close()
        best_validation_loss = write_summary_log( fp_summary, intensity_net_epoch_score_lst, au_name, phase_lst, network_name, best_epoch )

        best_validation_loss_lst[au_name].append(best_validation_loss)

fp_summary.close()






In [ ]:
# Get best trial
best_trial_lst = {}
for au_name in au_name_lst:
    best_trial = np.argmin(best_validation_loss_lst[au_name])
    best_trial_lst[au_name] = best_trial
best_trial_lst

In [ ]:
pair_net_best_filename_lst = {}
for au_name in au_name_lst:
    pair_net_best_filename_lst[au_name] = "./model/pair_net_%s_epoch_best_%s_trial_%d.pt" % (au_name, "posesall", best_trial_lst[au_name] )

# Predict pseudo-intensity(mini)
dataloaders_intensity_tune = get_dataloaders_for_intensity_tune( intensity_tune_dataset_path_lst, au_name_lst, [ phase.replace("Full","") for phase in phase_full_lst], batch_size_int, True )
predict_pairwise_net( pair_net_best_filename_lst, intensity_tune_dataset_path_lst, dataloaders_intensity_tune, au_name_lst, [ phase.replace("Full","") for phase in phase_full_lst], device, True )
memeory_free_dataloaders_intensity_tune( dataloaders_intensity_tune )

# Predict pseudo-intensity
dataloaders_intensity_tune = get_dataloaders_for_intensity_tune( intensity_tune_dataset_path_lst, au_name_lst, [ phase.replace("Full","") for phase in phase_full_lst], batch_size_int, False )
predict_pairwise_net( pair_net_best_filename_lst, intensity_tune_dataset_path_lst, dataloaders_intensity_tune, au_name_lst, [ phase.replace("Full","") for phase in phase_full_lst], device, False )
memeory_free_dataloaders_intensity_tune( dataloaders_intensity_tune )

intensity_net_best_filename_lst = {}
for au_name in au_name_lst:
    intensity_net_best_filename_lst[au_name] = "./model/intensity_net_%s_epoch_best_%s_trial_%d.pt" % (au_name, "posesall", best_trial_lst[au_name] )

    
fp = open("./model/intensity_net_param_lst_trial_%d.json" % best_trial_lst[au_name])
intensity_net_param_lst = json.load( fp )
fp.close()
    
# Predict intensity
make_dataset_for_intensity_net( intensity_tune_dataset_path_lst, intensity_conv_dataset_num_lst, dataset_lst_dirname, au_name_lst, [], phase_full_lst, False )
dataloaders_intensity_conv = get_dataloaders_for_intensity_net( intensity_tune_dataset_path_lst, au_name_lst, [], phase_full_lst, intensity_net_param_lst, batch_size_int, True )
predict_all_testset( dataloaders_intensity_conv, intensity_tune_dataset_path_lst, intensity_net_best_filename_lst, au_name_lst, device, phase_full_lst )
memory_free_intensity_net_dataloaders( dataloaders_intensity_conv )
